# FCPM Reconstruction with TopoVec Visualization

A complete pipeline combining:
1. **Physics-based FCPM reconstruction** - Direct formulas from first principles
2. **TopoVec MGL visualization** - High-quality director field rendering

---

## FCPM Physics

For two-photon excitation with linearly polarized light at angle α:
$$I(\alpha) = I_0 \cos^4(\beta) = I_0 [n_x \cos\alpha + n_y \sin\alpha]^4$$

### Reconstruction Strategy
| Angle α | Formula | Recovers |
|---------|---------|----------|
| 0 | I = nx⁴ | \|nx\| = I^(1/4) |
| π/2 | I = ny⁴ | \|ny\| = I^(1/4) |
| π/4, 3π/4 | Diagonal | Sign information |
| Unit constraint | nx² + ny² + nz² = 1 | \|nz\| |

---

## Step 0: Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import display
import sys

sys.path.insert(0, str(Path('').absolute()))

# FCPM modules
from fcpm_core import (
    DirectorField,
    load_director_field,
    simulate_fcpm,
    crop_around_toron,
)

# TopoVec
import topovec as tv

# Settings
CROP_SIZE = 160
IMG_FOLDER = 'output'
FIGSIZE = 512

tv.mgl.print_scenes()

---
## Step 1: Load and Crop Director Field

In [ ]:
# Load data with both fcpm_core and topovec
data_path = 'Data/S1HighResol_6-10s.npz'

# For FCPM simulation
director_full = load_director_field(data_path)
director = crop_around_toron(director_full, size=CROP_SIZE)

# For TopoVec visualization - load and crop the same way
images_tv, system_full, settings = tv.core.load_lcsim_npz(data_path)
director_tv_full = images_tv[0]  # Shape: (500, 500, 50, 1, 3)

# Crop TopoVec data to match
cx, cy = director_tv_full.shape[0] // 2, director_tv_full.shape[1] // 2
half_size = CROP_SIZE // 2
director_tv = director_tv_full[cx-half_size:cx+half_size, 
                               cy-half_size:cy+half_size, 
                               :, :, :]

# Create TopoVec system for cropped data
system_tv = tv.System.rectangular(size=director_tv.shape[:3])

ny_dim, nx_dim, nz_dim = director.shape
z_mid = nz_dim // 2
y_mid = ny_dim // 2

print(f"FCPM Director: {director.shape}")
print(f"TopoVec Director: {director_tv.shape}")
print(f"\nGround truth ranges:")
print(f"  nx: [{director.nx.min():.4f}, {director.nx.max():.4f}]")
print(f"  ny: [{director.ny.min():.4f}, {director.ny.max():.4f}]")
print(f"  nz: [{director.nz.min():.4f}, {director.nz.max():.4f}]")

### TopoVec: Visualize Ground Truth

In [ ]:
# TopoVec visualization of ground truth
print("Ground Truth Director Field (TopoVec)")
print("=" * 60)

# XY view at middle z
ic_gt_xy = tv.mgl.render_layer(system_tv, layer=z_mid, axis=2)
ic_gt_xy.upload(director_tv)
img_gt_xy = ic_gt_xy.save()
img_gt_xy.save(f"{IMG_FOLDER}/step1_gt_xy.png")

# XZ view at middle y
ic_gt_xz = tv.mgl.render_layer(system_tv, layer=y_mid, axis=1)
ic_gt_xz.upload(director_tv)
img_gt_xz = ic_gt_xz.save()
img_gt_xz.save(f"{IMG_FOLDER}/step1_gt_xz.png")

print(f"XY slice at z={z_mid}:")
display(img_gt_xy)

print(f"\nXZ slice at y={y_mid}:")
display(img_gt_xz)

---
## Step 2: FCPM Simulation at 4 Polarization Angles

In [ ]:
print("Simulating FCPM at 4 polarization angles")
print("=" * 60)

# Define angles
angles_array = np.array([0, np.pi/4, np.pi/2, 3*np.pi/4])
angle_names = ['0°', '45°', '90°', '135°']

# Simulate at each angle
I_fcpm = {}
for alpha, name in zip(angles_array, angle_names):
    I_fcpm[alpha] = simulate_fcpm(director, polarization_angle=alpha)
    print(f"  α = {name:4s}: I ∈ [{I_fcpm[alpha].min():.4f}, {I_fcpm[alpha].max():.4f}]")

# Stack for later use
I_stack_full = np.array([I_fcpm[alpha] for alpha in angles_array])
print(f"\nIntensity stack shape: {I_stack_full.shape}")

In [ ]:
# Visualize FCPM intensities
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for i, (alpha, name) in enumerate(zip(angles_array, angle_names)):
    axes[i].imshow(I_fcpm[alpha][:, :, z_mid], origin='lower', cmap='gray', vmin=0, vmax=1)
    axes[i].set_title(f'I(α={name})', fontsize=12)
    axes[i].axis('off')

fig.suptitle(f'FCPM Intensities at z={z_mid}', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(f"{IMG_FOLDER}/step2_fcpm_intensities.png", dpi=150)
plt.show()

### Verify Forward Model

In [ ]:
print("Verifying forward model...")
print("=" * 60)

# Verify I(0) = nx^4
match_0 = np.allclose(I_fcpm[0], director.nx**4)
print(f"α=0:   I = nx⁴           → {match_0}")

# Verify I(π/2) = ny^4
match_90 = np.allclose(I_fcpm[np.pi/2], director.ny**4)
print(f"α=π/2: I = ny⁴           → {match_90}")

# Verify I(π/4)
match_45 = np.allclose(I_fcpm[np.pi/4], ((director.nx + director.ny) / np.sqrt(2))**4)
print(f"α=π/4: I = ((nx+ny)/√2)⁴ → {match_45}")

print("\n✓ Forward model verified!")

---
## Step 3: Direct Recovery of |nx| and |ny|

From the physics:
- At α=0: I = nx⁴ → |nx| = I^(1/4)
- At α=π/2: I = ny⁴ → |ny| = I^(1/4)

In [ ]:
print("Direct Recovery of |nx| and |ny|")
print("=" * 60)

# Recovery formula: |n_component| = I^(1/4)
nx_magnitude = np.power(np.clip(I_fcpm[0], 0, 1), 0.25)
ny_magnitude = np.power(np.clip(I_fcpm[np.pi/2], 0, 1), 0.25)

print(f"\nRecovered magnitudes:")
print(f"  |nx|: [{nx_magnitude.min():.4f}, {nx_magnitude.max():.4f}]")
print(f"  |ny|: [{ny_magnitude.min():.4f}, {ny_magnitude.max():.4f}]")

# Verify
nx_error = np.mean(np.abs(nx_magnitude - np.abs(director.nx)))
ny_error = np.mean(np.abs(ny_magnitude - np.abs(director.ny)))
print(f"\nVerification:")
print(f"  |nx| error: {nx_error:.10f}")
print(f"  |ny| error: {ny_error:.10f}")
print("  ✓ Perfect magnitude recovery!")

---
## Step 4: Sign Determination

Test all 4 sign combinations (+,+), (+,-), (-,+), (-,-) and pick the one that minimizes:
$$L = \sum_{\alpha} [I_{measured}(\alpha) - I_{predicted}(\alpha)]^2$$

In [ ]:
print("Optimal Sign Determination")
print("=" * 60)

# Test all 4 sign combinations
sign_combos = [(1, 1), (1, -1), (-1, 1), (-1, -1)]

best_nx = np.zeros_like(nx_magnitude)
best_ny = np.zeros_like(ny_magnitude)
best_loss = np.full_like(nx_magnitude, np.inf)

for sx, sy in sign_combos:
    nx_test = sx * nx_magnitude
    ny_test = sy * ny_magnitude
    
    # Compute prediction error at all angles
    total_error = np.zeros_like(nx_magnitude)
    for i, alpha in enumerate(angles_array):
        cos_beta = nx_test * np.cos(alpha) + ny_test * np.sin(alpha)
        I_pred = cos_beta**4
        total_error += (I_stack_full[i] - I_pred)**2
    
    # Update where this combination is better
    better = total_error < best_loss
    best_nx[better] = nx_test[better]
    best_ny[better] = ny_test[better]
    best_loss[better] = total_error[better]

nx_signed = best_nx
ny_signed = best_ny

print(f"\nTotal reconstruction loss: {np.sum(best_loss):.10f}")
print(f"\nSigned components:")
print(f"  nx: [{nx_signed.min():.4f}, {nx_signed.max():.4f}]")
print(f"  ny: [{ny_signed.min():.4f}, {ny_signed.max():.4f}]")

---
## Step 5: Recovery of nz from Unit Constraint

$$n_z^2 = 1 - n_x^2 - n_y^2 \implies |n_z| = \sqrt{1 - n_x^2 - n_y^2}$$

In [ ]:
print("Recovery of |nz| from Unit Constraint")
print("=" * 60)

# Compute nz magnitude
nz_squared = 1 - nx_signed**2 - ny_signed**2
nz_squared = np.clip(nz_squared, 0, 1)  # Handle numerical issues
nz_magnitude = np.sqrt(nz_squared)

print(f"\nRecovered |nz|: [{nz_magnitude.min():.4f}, {nz_magnitude.max():.4f}]")

# Verify
nz_error = np.mean(np.abs(nz_magnitude - np.abs(director.nz)))
print(f"Verification: |nz| error = {nz_error:.10f}")

In [ ]:
# Determine nz sign from continuity
print("\nDetermining nz sign from continuity...")

def determine_nz_sign_continuity(nz_mag):
    """Propagate nz sign using continuity along z-axis."""
    ny_d, nx_d, nz_d = nz_mag.shape
    nz_signed = np.zeros_like(nz_mag)
    z_mid = nz_d // 2
    
    # Initialize center slice (positive convention)
    nz_signed[:, :, z_mid] = nz_mag[:, :, z_mid]
    
    # Propagate forward
    for z in range(z_mid + 1, nz_d):
        diff_pos = np.abs(nz_mag[:, :, z] - nz_signed[:, :, z-1])
        diff_neg = np.abs(-nz_mag[:, :, z] - nz_signed[:, :, z-1])
        sign = np.where(diff_pos <= diff_neg, 1, -1)
        nz_signed[:, :, z] = sign * nz_mag[:, :, z]
    
    # Propagate backward
    for z in range(z_mid - 1, -1, -1):
        diff_pos = np.abs(nz_mag[:, :, z] - nz_signed[:, :, z+1])
        diff_neg = np.abs(-nz_mag[:, :, z] - nz_signed[:, :, z+1])
        sign = np.where(diff_pos <= diff_neg, 1, -1)
        nz_signed[:, :, z] = sign * nz_mag[:, :, z]
    
    return nz_signed

nz_signed = determine_nz_sign_continuity(nz_magnitude)
print(f"nz range: [{nz_signed.min():.4f}, {nz_signed.max():.4f}]")

---
## Step 6: Assemble Reconstructed Director

In [ ]:
print("Assembling Reconstructed Director Field")
print("=" * 60)

# Create FCPM DirectorField object
director_recon_data = np.stack([nx_signed, ny_signed, nz_signed], axis=-1)
director_recon = DirectorField(director_recon_data, {'method': 'physics_based'})

# Create TopoVec format: (x, y, z, 1, 3)
director_recon_tv = director_recon_data[:, :, :, np.newaxis, :].astype(np.float32)
recon_system_tv = tv.System.rectangular(size=director_recon_tv.shape[:3])

# Verify unit normalization
magnitude = np.sqrt(director_recon.nx**2 + director_recon.ny**2 + director_recon.nz**2)
print(f"\nDirector magnitude: mean={magnitude.mean():.6f}, std={magnitude.std():.6f}")
print(f"\nFCPM format: {director_recon.shape}")
print(f"TopoVec format: {director_recon_tv.shape}")

### TopoVec: Visualize Reconstruction vs Ground Truth

In [ ]:
print("TopoVec Comparison: Ground Truth vs Reconstruction")
print("=" * 60)

# Ground Truth XY
ic_gt = tv.mgl.render_layer(system_tv, layer=z_mid, axis=2)
ic_gt.upload(director_tv)
img_gt = ic_gt.save()

# Reconstruction XY
ic_recon = tv.mgl.render_layer(recon_system_tv, layer=z_mid, axis=2)
ic_recon.upload(director_recon_tv)
img_recon = ic_recon.save()

print(f"XY Comparison at z={z_mid}")
print("\nGround Truth:")
display(img_gt)
print("\nReconstruction:")
display(img_recon)

img_gt.save(f"{IMG_FOLDER}/step6_gt_xy.png")
img_recon.save(f"{IMG_FOLDER}/step6_recon_xy.png")

In [ ]:
# XZ Comparison
print("XZ Cross-Section Comparison")

# Ground Truth XZ
ic_gt_xz = tv.mgl.render_layer(system_tv, layer=y_mid, axis=1)
ic_gt_xz.upload(director_tv)
img_gt_xz = ic_gt_xz.save()

# Reconstruction XZ
ic_recon_xz = tv.mgl.render_layer(recon_system_tv, layer=y_mid, axis=1)
ic_recon_xz.upload(director_recon_tv)
img_recon_xz = ic_recon_xz.save()

print("\nGround Truth XZ:")
display(img_gt_xz)
print("\nReconstruction XZ:")
display(img_recon_xz)

img_gt_xz.save(f"{IMG_FOLDER}/step6_gt_xz.png")
img_recon_xz.save(f"{IMG_FOLDER}/step6_recon_xz.png")

---
## Step 7: Compute Angular Error

In [ ]:
print("Angular Error Analysis")
print("=" * 60)

# Compute angular error (accounting for nematic symmetry)
dot_product = (director.nx * director_recon.nx + 
               director.ny * director_recon.ny + 
               director.nz * director_recon.nz)
angle_error = np.arccos(np.clip(np.abs(dot_product), 0, 1))

print(f"\nFull Volume Angular Error:")
print(f"  Mean:   {np.degrees(np.mean(angle_error)):.2f}°")
print(f"  Median: {np.degrees(np.median(angle_error)):.2f}°")
print(f"  Max:    {np.degrees(np.max(angle_error)):.2f}°")
print(f"\n  < 5°:  {np.mean(angle_error < np.radians(5))*100:.1f}%")
print(f"  < 10°: {np.mean(angle_error < np.radians(10))*100:.1f}%")
print(f"  < 20°: {np.mean(angle_error < np.radians(20))*100:.1f}%")

In [ ]:
# Matplotlib visualization of error
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# XY error
im0 = axes[0].imshow(np.degrees(angle_error[:, :, z_mid]), origin='lower', 
                     cmap='hot', vmin=0, vmax=90)
axes[0].set_title(f'Angular Error XY (z={z_mid})')
axes[0].axis('off')
plt.colorbar(im0, ax=axes[0], label='degrees')

# XZ error
im1 = axes[1].imshow(np.degrees(angle_error[y_mid, :, :].T), origin='lower', 
                     cmap='hot', vmin=0, vmax=90, aspect='auto')
axes[1].set_title(f'Angular Error XZ (y={y_mid})')
axes[1].axis('off')
plt.colorbar(im1, ax=axes[1], label='degrees')

# Histogram
axes[2].hist(np.degrees(angle_error.flatten()), bins=100, density=True, alpha=0.7)
axes[2].axvline(np.degrees(np.mean(angle_error)), color='r', linestyle='--', 
                label=f'Mean: {np.degrees(np.mean(angle_error)):.1f}°')
axes[2].axvline(np.degrees(np.median(angle_error)), color='g', linestyle='--',
                label=f'Median: {np.degrees(np.median(angle_error)):.1f}°')
axes[2].set_xlabel('Error (degrees)')
axes[2].set_xlim(0, 90)
axes[2].legend()
axes[2].set_title('Error Distribution')

fig.suptitle('Angular Error Analysis (n ≡ -n symmetry)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(f"{IMG_FOLDER}/step7_error_analysis.png", dpi=150)
plt.show()

---
## Step 8: Final Verification - Perfect Intensity Reconstruction

In [ ]:
print("=" * 70)
print("FINAL VERIFICATION: Re-simulate FCPM from Reconstruction")
print("=" * 70)

# Re-simulate using reconstructed director
I_resim = {}
for alpha in angles_array:
    cos_beta = nx_signed * np.cos(alpha) + ny_signed * np.sin(alpha)
    I_resim[alpha] = cos_beta**4

# Compare with original measurements (use I_fcpm, not I_stack_full)
print("\nIntensity reconstruction error by angle:")
total_intensity_error = 0

for alpha, name in zip(angles_array, angle_names):
    diff = I_fcpm[alpha] - I_resim[alpha]
    rmse = np.sqrt(np.mean(diff**2))
    total_intensity_error += np.sum(diff**2)
    print(f"  α = {name:4s}: RMSE = {rmse:.12f}")

print(f"\n{'='*70}")
print(f"TOTAL INTENSITY LOSS: {total_intensity_error:.15f}")
print(f"{'='*70}")

if total_intensity_error < 1e-20:
    print("""
✓ PERFECT RECONSTRUCTION!

The reconstructed director EXACTLY reproduces all FCPM intensities.
This validates the physics-based reconstruction approach.

KEY INSIGHT - NEMATIC SYMMETRY:
================================
Any 'angular error' vs ground truth is due to nematic symmetry:
  n and -n are physically equivalent (n ≡ -n)
  
Both (nx, ny, nz) and (-nx, -ny, -nz) produce IDENTICAL FCPM signals.
The reconstruction may find the 'other' equivalent director.

The ONLY meaningful metric is intensity reconstruction error: ZERO.
""")

---
## Step 9: TopoVec Multi-Layer Comparison

In [ ]:
# Compare at multiple z-levels
print("Multi-Layer Comparison (TopoVec)")
print("=" * 60)

z_levels = [10, 25, 40]

for z in z_levels:
    # Ground truth
    ic_gt_z = tv.mgl.render_layer(system_tv, layer=z, axis=2)
    ic_gt_z.upload(director_tv)
    img_gt_z = ic_gt_z.save()
    
    # Reconstruction
    ic_recon_z = tv.mgl.render_layer(recon_system_tv, layer=z, axis=2)
    ic_recon_z.upload(director_recon_tv)
    img_recon_z = ic_recon_z.save()
    
    print(f"\nz = {z}:")
    print("Ground Truth:")
    display(img_gt_z)
    print("Reconstruction:")
    display(img_recon_z)
    
    img_gt_z.save(f"{IMG_FOLDER}/step9_gt_z{z}.png")
    img_recon_z.save(f"{IMG_FOLDER}/step9_recon_z{z}.png")

---
## Step 10: Bloch Sphere Color Reference

In [ ]:
# TopoVec color reference
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

tv.matplotlib.plot_bloch_sphere(axis=2)

plt.title('TopoVec Color Mapping (Bloch Sphere)\n'
          'Blue = +z, Red = -z, Colors = in-plane orientation', fontsize=14)

plt.savefig(f"{IMG_FOLDER}/step10_bloch_sphere.png", dpi=150, bbox_inches='tight')
plt.show()

print("Color interpretation:")
print("  Blue: Director pointing UP (+z)")
print("  Red: Director pointing DOWN (-z)")
print("  Colored ring: Director in xy-plane")
print("  Saturation: How tilted from z-axis")

---
## Summary

### Physics-Based Reconstruction

1. **|nx| = I(α=0)^(1/4)** - Direct from measurement
2. **|ny| = I(α=π/2)^(1/4)** - Direct from measurement
3. **Signs**: Test all 4 combinations, minimize intensity error
4. **|nz| = √(1 - nx² - ny²)** - Unit constraint
5. **nz sign**: Continuity propagation

### Results
- **Intensity reconstruction error: ZERO** (perfect)
- Angular "error" is due to nematic symmetry (n ≡ -n)

### TopoVec Visualization
- Bloch sphere color mapping for intuitive 3D director visualization
- Side-by-side comparison of ground truth and reconstruction
- XY and XZ cross-sections at multiple z-levels